## 1. Import libraries

In [1]:
import numpy as np
import pandas as pd

from scipy.fft import fft, ifft

import matplotlib.pyplot as plt

import bokeh
from bokeh.plotting import figure,show,output_notebook
from bokeh.io import show

output_notebook()

Loading BokehJS ...

## 2. Read CSV files

In [2]:
df = pd.read_csv('pr_dataset.csv')
df.index=pd.to_datetime(df['Date']).dt.date


df

,Date,pr
Date,,
2010-10-20,2010-10-20 00:00:00+09:30,0.847736
2010-10-21,2010-10-21 00:00:00+09:30,0.814093
2010-10-22,2010-10-22 00:00:00+09:30,0.808109
2010-10-23,2010-10-23 00:00:00+09:30,0.814994
2010-10-24,2010-10-24 00:00:00+09:30,0.828574
...,...,...
2013-10-16,2013-10-16 00:00:00+09:30,0.760520
2013-10-17,2013-10-17 00:00:00+09:30,0.802566
2013-10-18,2013-10-18 00:00:00+09:30,0.771059


In [3]:
p1=figure(title=None,x_axis_type='datetime',height=400,y_range=[0.5,1.2])
p1.scatter(df.index,(df['pr']),color='red',legend_label='PR')

show(p1)

### frequency

In [4]:
#frequency of data
n=len(df['pr'])
freq= (1/(n))* np.arange(n)

L = np.arange(1,np.floor(n/2),dtype='int') 

## 4. PR

### a.Fast Fourier Transform

In [5]:
# determinig FFT of PR 

fft_pr_ = fft(df['pr'].values,n)   
# fft_pr_ are complex numbers. 

PSD_pr=np.abs(fft_pr_)
# PSD is the Power Spectral Density of PR values
# The PSD is represented for half of the frequency as the PSD would be distributed symmetric to x axis

In [7]:
p2=figure(title=None,height=400)
p2.line(freq[L],PSD_pr[L],color='red')
p2.scatter(freq[L],PSD_pr[L],color='red',legend_label='FFT PR')

show(p2)

### b. filter

In [8]:
#indicies corresponding to seasonal and residue components
# Note- The Power Spectral Density (PSD) limits will vary according to the PSD derived from the dataset. 
# This step can be automated to adjust based on the specific dataset being analysed.
indicies_pr_seasonal = (PSD_pr >20 ) #seasonal component corresponding to periodic seasonal frequency
indicies_pr_residue = (PSD_pr <20 ) #residuae component- dataset after elimination of seasonal component

# PSD corresponding to seasonal and residue components
#PSD_clean =PSD * indicies #zero out all other values
PSD_pr_clean_seasonal =PSD_pr * indicies_pr_seasonal   #zero out all other values
PSD_pr_clean_residue =PSD_pr * indicies_pr_residue   #zero out all other values

# frequency corresponding to seasonal and residue components
freq_seasonal= freq* indicies_pr_seasonal
freq_residue= freq* indicies_pr_residue

# FFT of PR corresponding to seasonal and residue components
fft_pr_clean_seasonal_ = indicies_pr_seasonal * fft_pr_ #complex numbers
fft_pr_clean_residue_ = indicies_pr_residue * fft_pr_ #complex numbers

In [10]:
p3=figure(title=None,height=400)
p3.line(freq[L],np.real(PSD_pr[L]),color='red',legend_label='PSD')
p3.scatter(freq[L],PSD_pr[L],color='red',legend_label='PSD')
p3.scatter(freq[L],PSD_pr_clean_seasonal[L],color='green',legend_label='PSD - seasonal component')
p3.scatter(freq[L],PSD_pr_clean_residue[L],color='blue',legend_label='PSD - residue component')


show(p3)

### c. Inverse of Fast Fourier Transform (IFFT)

In [11]:
ifft_pr_seasonal =ifft(fft_pr_clean_seasonal_)  # ifft of seasonal component corresponding to periodic seasonal frequency
ifft_pr_residue =ifft(fft_pr_clean_residue_)  # ifft of residue component- dataset after elimination of seasonal component

In [12]:
df['pr_seasonal'] = np.real(ifft_pr_seasonal)
df['pr_residue'] = np.real(ifft_pr_residue)

In [13]:
p4=figure(title=None,x_axis_type='datetime',height=400,y_range=[0.5,1.2])
p4.scatter(df.index,(df['pr']),color='red',legend_label='PR')
p4.line(df.index,(df['pr_seasonal']),color='black',legend_label='Seasonal component of PR',line_width=3)

show(p4)

In [17]:
pr_min=df['pr_seasonal'].min()
pr_max=df['pr_seasonal'].max()
pr_h=((pr_max-pr_min)/2)+pr_min
pr_h

0.812272040588348

In [21]:
p5=figure(title=None,x_axis_type='datetime',height=400,y_range=[0.5,1.2])

p5.scatter(df.index,(df['pr_residue']+pr_h),color='blue',legend_label='Residue component of PR')

show(p5)

In [ ]:
df.to_csv(pr_dataset.csv)